In [10]:
#Importing the libraries to be used:
import sklearn
import numpy as np
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, preprocessing
import numpy.polynomial.polynomial as poly
%matplotlib inline

ModuleNotFoundError: No module named 'sklearn'

In [4]:
df = pd.read_csv('creditcard.csv')
print(df.shape)

(284807, 31)


In [5]:
df.dropna('columns')
print(df.shape)

(284807, 31)
